In [ ]:
# default_exp readproc

# readproc

This module can be used to generate '.csv' files containing counts for each read in a '.fasta' file.

> API details



In [ ]:
#hide
from nbdev.showdoc import *



In [ ]:
#export
import re
from pathlib import Path
import pandas as pd


def trim_cterm(inputFile, outputFile, motifToTrim):
    with open(inputFile, 'r') as inputFH:
        with open(outputFile,'w') as outputFH:
            i = 0
            for line in inputFH:
                i += 1
                if i%2 ==1: outputFH.write(line)
                else:
                    line = re.sub(motifToTrim, '', line)
                    #line += '\n'
                    outputFH.write(line)

                    
def fasta_to_counts(inputFile):
    
    
    df_seq_counts = pd.DataFrame()
    cur_name = ''
    cur_seq = ''
    dict_temp_seqs = {}
    i = 0
    #make a temporary dict for counting seqs
    with open(inputFile, 'r') as inputFH:
        for line in inputFH:
            #print(i)
            if line[0] == '>': # lines starting with '>' are seq names 
                if cur_name != '': #add previous seq to dict, if it exists
                    dict_temp_seqs = increment_seq_counts(dict_temp_seqs, cur_name, cur_seq)
                    #if cur_name in dict_temp_seqs.keys():
                    #    dict_temp_seqs[cur_name][1] += 1 # add 1 to count for cur_seq
                    #else:
                    #    dict_temp_seqs[cur_name] = [cur_seq, 1]
                cur_name = line.rstrip()[1:] # strip newline and '>'
                cur_seq = ''
                i += 1
            else: # odd lines are the actual sequences, concatenate until a new sequence starts
                cur_seq += line.rstrip()
               # print(cur_seq)
        dict_temp_seqs = increment_seq_counts(dict_temp_seqs, cur_name,cur_seq)
        #if cur_name in dict_temp_seqs.keys(): #add the last seq to the dict
        #    dict_temp_seqs[cur_name][1] += 1
        #else:
        #    dict_temp_seqs[cur_name] =  [cur_seq, 1]
            
                
    return dict_temp_seqs

def increment_seq_counts(seq_dict, cur_seq_name, cur_seq):
    if cur_seq_name in seq_dict.keys():
        seq_dict[cur_seq_name][0] +=1
    else:
        seq_dict[cur_seq_name] = [1, cur_seq]
    return seq_dict


def dict_to_df(seq_dict):
    dict_for_df={}
    for i, cur_name in enumerate(seq_dict):
        cur_count = seq_dict[cur_name][0]
        cur_seq = seq_dict[cur_name][1]
        cur_seqZ = 'Z' + cur_seq + 'Z' 
        dict_for_df[i] = [cur_name, seq_dict[cur_name][0],cur_seq, cur_seqZ]
    df_seqs = pd.DataFrame.from_dict(dict_for_df, orient='index', columns=['seq_name', 'count', 'seq', 'seqZ'])
    return df_seqs
    
    
#def df_to_csv:

    
    #make a second dic with the final counts and also the sequences with terminal Zs, to build the DF



# Testing

## trim_cterm

In [ ]:
test_input_path = Path.cwd()/'tests/test-trim-in.fasta'
test_trimmed_output_path = Path.cwd()/'tests/test-trim-out.fasta'
test_cterm_pattern_to_trim = 'LNEAIP.*$'

trim_cterm(test_input_path, test_trimmed_output_path, test_cterm_pattern_to_trim)

with open(test_trimmed_output_path, 'r') as test_FH:
    for i,line in enumerate(test_FH):
        if i == 0:
            assert line.rstrip() == '>123'
        if i == 1:
            assert line.rstrip() == 'CCC'
        if i == 2:
            assert line.rstrip() == '>128378'
        if i == 3:
            assert line.rstrip() == 'CCAC'
    
    assert i == 3

## fasta_to_counts

In [ ]:
test_count_input_path = Path.cwd()/'tests/test-count-in.fasta' 
test_dict = fasta_to_counts(test_count_input_path)
df_seqs = dict_to_df(test_dict)

assert df_seqs['seq_name'][0] == '123'
assert df_seqs['seq_name'][1] == '128378'
assert df_seqs['count'][0]    == 2
assert df_seqs['count'][1]    == 3
assert df_seqs['seq'][0]      == 'CCC'
assert df_seqs['seq'][1]      == 'CCAC'
assert df_seqs['seqZ'][0]     == 'ZCCCZ'
assert df_seqs['seqZ'][1]     == 'ZCCACZ'
